In [6]:
import pandas as pd
import numpy as np
from process_and_ml.data_wrangler import load_test_file

In [7]:
from config import settings
path_test_udacity = '../data/train_test_files/Udacity_MAILOUT_052018_TEST.csv'

In [8]:
test_df_compare, lnr = load_test_file(path_test_udacity)

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
customers = pd.read_csv('../data/cleaned_data/customers_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)

In [10]:
#population = pd.read_csv('../data/cleaned_data/azdias_clean.csv', index_col=[0]).sample(frac=0.4, random_state=settings.RANDOM_STATE)
df_info_columns = pd.read_excel('/home/jair/Documents/arvato_project/data/dataframe_info/DIAS Information Levels - Attributes 2017.xlsx', engine='openpyxl')
column_array = df_info_columns.iloc[0].values
column_array[0] = 'to_drop'
df_info_columns.columns = column_array

df_info_columns.drop('to_drop', axis=1, inplace=True)
df_info_columns = df_info_columns.iloc[1:]

In [11]:
df_column_attributes = pd.read_excel('/home/jair/Documents/arvato_project/data/dataframe_info/DIAS Attributes - Values 2017.xlsx', engine='openpyxl')
column_array = df_column_attributes.iloc[0].values
column_array[0] = 'to_drop'
df_column_attributes.columns = column_array
df_column_attributes.drop('to_drop', axis=1, inplace=True)
df_column_attributes =df_column_attributes.iloc[1:]


In [12]:
columns_with_info = df_info_columns.Attribute.unique()
columns_with_attributes = df_column_attributes.Attribute.unique()

In [13]:
columns_with_attributes = np.delete(columns_with_attributes, 1)
columns_with_info = np.unique(np.concatenate((columns_with_info, columns_with_attributes)))

In [14]:
columns_with_attributes = [col for col in columns_with_info if col  in test_df]


In [15]:
cod_nan = pd.read_csv('/home/jair/Documents/arvato_project/data/wrangler/azdias_costumer_to_nan.csv')

In [16]:
cod_nan.drop(cod_nan.columns[0], axis=1, inplace=True)


In [17]:
cod_nan.set_index('Attribute', inplace=True)

In [18]:
dict_to_nan = cod_nan.to_dict()['Value']
for key, value in dict_to_nan.items():
    if len(value)> 2:
        dict_to_nan[key] = [int(num) for num in value.split(',')]
    else:
        dict_to_nan[key] = int(value)

In [22]:
dict_to_nan['CAMEO_DEUG_2015'] = [-1,'X']
dict_to_nan['CAMEO_DEU_2015'] = [-1,'XX']
dict_to_nan['CAMEO_DEUINTL_2015'] = [-1,'XX']



In [20]:
bip_flag = dict_to_nan.pop('BIP_FLAG')

In [24]:
df_test_cols = test_df.columns
for key, value, in dict_to_nan.items():
    if key in df_test_cols:
        test_df[key] = test_df[key].replace(value, np.nan)

In [25]:
test_df_compare, lnr = load_test_file(path_test_udacity)
# create cols
from process_and_ml.feature_engineer import fs_for_cat_part_one, fs_for_cat_part_two


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [26]:
checking_test = fs_for_cat_part_one(test_df)


In [27]:
checking_test = fs_for_cat_part_two(checking_test)

In [28]:
last_test = checking_test[customers.columns]

In [29]:
# savind test file for finally catboost prediction
last_test['LNR'] = lnr.values

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
last_test.to_csv('../data/cleaned_data/test_correct.csv', index=False)